
# Arguments

In [ ]:
num_of_epochs = 25
batch_size = 32
dataset_name = 'plantVillage-tomato-mohanty'
dataset_path = '../datasets/' + dataset_name
model_save_path = 'vgg-model.h5'

checkpoint_path = 'warmup-checkpoints'


model_diagram_path = 'vgg_model_diagram.png'

input_width = 224
input_height = 224
input_depth = 3

# Get classes
import os
import re
classes = os.listdir(dataset_path)
class_names = []

for i in classes:
    if(re.search("Tomato___", i)):
        class_names.append(i)
    
print('Classes: ', class_names)
print(len(class_names))


# Load model

In [ ]:
from keras.applications import VGG16
from keras.utils.vis_utils import plot_model


# Load pretrianed VGG model with FC layers removed
# explicitly deﬁne the input_tensor to be 224×224×3 pixels
base_model = VGG16(weights='imagenet',include_top=False,
                  input_tensor=Input(shape = (input_width,input_height, input_depth)))

plot_save_path = 'vgg16-base-model.png'
plot_model(base_model, to_file=model_diagram_path, show_shapes=True)

# Custom FC layer

In [ ]:
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense

head_model = baseModel.output
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(D,activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
# Add a softmaxc layer
head_model = Dense(len(class_names),activation='softmax')(head_model)

plot_save_path = 'custom-head.png'
plot_model(head_model, to_file=model_diagram_path, show_shapes=True)

# Attach custom head to model

In [ ]:


# place the head FC model on top of the base model 
model = Model(inputs=base_model.input, outputs = head_model)

# freeze weights in base model
for layer in base_model.layers:
    layer.trainable = False

plot_save_path = 'full-model.png'
plot_model(model, to_file=model_diagram_path, show_shapes=True)

# Compile model

In [ ]:
from keras.optimizers import RMSprop

print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)

model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# Since we only train the new fully connected layer, we do a few iterations
print("[INFO] training head...")

# Load data

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# Load image paths
print("[INFO] loading images...")
image_paths = list(paths.list_images(args['dataset']))

# Initial image preprocessing
aap = AspectAwarePreprocesser(input_width, input_height)
iap= ImageToArrayPreprocess()

#Load image data and perform image data preprocessing
sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
(data,labels)  = sdl.load(image_paths,verbose=500)
data = data.astype("float") / 255.0


# train test split
(train_x,text_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(train_y)
testY = LabelBinarizer().fit_transform(test_y)



# Warm up head

In [ ]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]

In [ ]:


model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(trainX) //32,
             verbose = 1,
             callbacks=callbacks)

# Evaluate after warmup

In [ ]:
# 
print("[INFO] evaluating after initialization...")
predictions = model.predict(text_x,batch_size=32)
print(classification_report(text_y.argmax(axis =1),
                            predictions.argmax(axis =1),target_names=classNames))